In [32]:
from pprint import pprint
import numpy as np
from sklearn.linear_model import LinearRegression
import mlflow



# enable autologging
mlflow.sklearn.autolog()

model = LinearRegression()
with mlflow.start_run() as run:
    model.fit(X, y)

# fetch logged data
params, metrics, tags, artifacts = fetch_logged_data(run.info.run_id)


NameError: name 'X' is not defined

In [33]:
def fetch_logged_data(run_id):
    client = mlflow.tracking.MlflowClient()
    data = client.get_run(run_id).data
    tags = {k: v for k, v in data.tags.items() if not k.startswith("mlflow.")}
    artifacts = [f.path for f in client.list_artifacts(run_id, "model")]
    return {"params":data.params, "metrices":data.metrics,"tags":tags}

In [35]:
import vaex

from goldilox.datasets import load_iris
from vaex.ml.sklearn import Predictor
from sklearn.linear_model import LogisticRegression

df, features, target = load_iris()
train, test = vaex.from_pandas(df).ml.train_test_split()


model = Predictor(model=LogisticRegression(),
                  features=features,
                        target=target,
                        prediction_name='lgbm')


with mlflow.start_run() as run:
    model.fit(train)
    metrics = mlflow.sklearn.eval_and_log_metrics(model.model, test[features], test[target].values, prefix="val_")

train = model.transform(train)
print(train.head(2))

/Users/yonatanalexander/development/xdss/goldilox/.venv/lib/python3.7/site-packages/vaex/ml/__init__.py:31: UserWarning: Make sure the DataFrame is shuffled
  warnings.warn('Make sure the DataFrame is shuffled')
/Users/yonatanalexander/development/xdss/goldilox/.venv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
2022/02/06 18:30:11 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/yonatanalexander/development/xdss/goldilox/.venv/lib/python3.7/site-packages/sklearn/utils/deprecation.py:

  #    sepal_length    sepal_width    petal_length    petal_width    target    lgbm
  0             4.8            3.1             1.6            0.2         0       0
  1             5.4            3.4             1.5            0.4         0       0


/Users/yonatanalexander/development/xdss/goldilox/.venv/lib/python3.7/site-packages/sklearn/base.py:439: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [37]:
from goldilox import Pipeline

pipeline = Pipeline.from_vaex(train, variables=fetch_logged_data(run.info.run_id))

In [25]:
pipeline.exaport_mlflow('mlflow_test')

AttributeError: 'VaexPipeline' object has no attribute 'exaport_mlflow'

In [20]:
mlflow_pyfunc_model_path = "mlflow_test"
mlflow.pyfunc.save_model(path=mlflow_pyfunc_model_path, python_model=GoldiloxWrapper(), artifacts=artifacts,conda_env=env)



In [21]:
!mlflow models serve -m mlflow_test --no-conda

2022/02/05 12:36:05 INFO mlflow.models.cli: Selected backend for flavor 'python_function'
2022/02/05 12:36:05 INFO mlflow.pyfunc.backend: === Running command 'gunicorn --timeout=60 -b 127.0.0.1:5000 -w 1 ${GUNICORN_CMD_ARGS} -- mlflow.pyfunc.scoring_server.wsgi:app'
[2022-02-05 12:36:06 +0100] [31079] [INFO] Starting gunicorn 20.1.0
[2022-02-05 12:36:06 +0100] [31079] [INFO] Listening at: http://127.0.0.1:5000 (31079)
[2022-02-05 12:36:06 +0100] [31079] [INFO] Using worker: sync
[2022-02-05 12:36:06 +0100] [31082] [INFO] Booting worker with pid: 31082
^C
[2022-02-05 13:00:03 +0100] [31079] [INFO] Handling signal: int
[2022-02-05 13:00:03 +0100] [31082] [INFO] Worker exiting (pid: 31082)


In [ ]:
# Create a Conda environment for the new MLflow Model that contains all necessary dependencies.
import cloudpickle


# Save the MLflow Model
mlflow_pyfunc_model_path = "xgb_mlflow_pyfunc"
mlflow.pyfunc.save_model(path=mlflow_pyfunc_model_path, python_model=XGBWrapper(), artifacts=artifacts, conda_env=env)

# Load the model in `python_function` format
loaded_model = mlflow.pyfunc.load_model(mlflow_pyfunc_model_path)

# Evaluate the model
import pandas as pd
test_predictions = loaded_model.predict(pd.DataFrame(x_test))
print(test_predictions)

